In [1]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/iisc/reverse_dictionary/iitp-movie-reviews/hi/hi-train.csv")
df.columns.values[0] = 'experience'
df.columns.values[1] = 'text'
df.head()

,experience,text
0,positive,’उड़ान’ से विक्रमादित्य\nमोटवाने\nने अच्छे सिन...
1,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...
2,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...
3,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...
4,positive,इन फुकरों से आप पहले भी मिल चुके हैं और आपने इ...


In [4]:
!pip3 install pandas_ods_reader
from pandas_ods_reader import read_ods
newdf=read_ods("/content/drive/MyDrive/iisc/reverse_dictionary/hi_3500.ods",headers=False)
newdf.columns.values[0]='text'
newdf.columns.values[1]='label'
newdf.head()
newdf.label=pd.Categorical(newdf.label)
newdf['label']=newdf['label'].cat.codes
newdf = newdf.dropna()
newdf.head()

,text,label
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,0
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,0
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,0
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,0
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,0


In [5]:
label=newdf['label']
input=newdf['text']
#label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}

# Map string labels to numeric labels
#label = label.map(label_mapping)
Y = pd.get_dummies(newdf['label']).values
print(Y)
onehot=Y

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]


In [6]:
tokenizer = AutoTokenizer.from_pretrained('monsoon-nlp/hindi-bert')
model = AutoModel.from_pretrained('monsoon-nlp/hindi-bert')

avg_embeddings_list = []

# Process each sentence in the DataFrame
for sentence in input:
    # Tokenize the input text
    tokens = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, padding=True, return_tensors='pt', max_length=128)

    # Forward pass to obtain embeddings
    with torch.no_grad():
        embeddings = model(tokens)[0]

    # Convert embeddings to a numpy array
    embeddings = embeddings.cpu().numpy()

    # Calculate the average embedding for each word
    avg_embeddings = np.mean(embeddings, axis=1)
    avg_embeddings_list.append(avg_embeddings)

# Stack the average embeddings for all sentences
avg_embeddings_matrix = np.vstack(avg_embeddings_list)

# Print the shape of the average embeddings matrix
print(avg_embeddings_matrix.shape)

(9077, 256)


In [7]:
x_train=avg_embeddings_matrix
y_train=Y
print(x_train.shape,y_train.shape)

(9077, 256) (9077, 3)


In [8]:
x_train,x_test,y_train,y_test=train_test_split(avg_embeddings_matrix,Y,train_size=0.7,random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Flatten,LSTM,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [10]:
def create_dan_model():
    model_dan = Sequential()
    model_dan.add(Dense(300, activation='linear', input_dim=256))
    model_dan.add(Dense(100, activation='elu'))
    model_dan.add(Dense(100, activation='elu'))
    model_dan.add(Dense(100, activation='elu'))
    model_dan.add(Dense(3, activation='softmax'))
    return model_dan

# Create the model
model_dan = create_dan_model()


In [11]:
print(x_train,y_train)

[[ 1.1272625e-03 -2.6613193e-02  1.2697988e+00 ...  3.4698851e-03
   4.6843704e-02  1.6145490e+00]
 [ 1.1613622e-03 -6.4509481e-02  1.0178461e+00 ...  3.4127417e-03
   3.3048816e-02  1.0789322e+00]
 [ 1.0932828e-03 -9.1597341e-02  1.1196188e+00 ...  3.7147477e-03
   2.4738364e-02  7.6778847e-01]
 ...
 [ 7.9488917e-04 -5.2268263e-02 -6.9373941e-01 ...  3.7798858e-03
   3.1013776e-02 -1.3831125e-01]
 [ 1.0243228e-03 -5.1807001e-02  1.8729410e+00 ...  3.6290621e-03
  -3.3208184e-04  1.2115167e+00]
 [ 1.0563293e-03 -4.3814436e-02  7.6423651e-01 ...  3.4554100e-03
   5.7709895e-02  8.8481432e-01]] [[0 1 0]
 [0 0 1]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [1 0 0]]


In [12]:
model_dan.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model_dan.summary()
model_dan.fit(x_train, y_train, epochs=200, batch_size=64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               77100     
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 3)                 303       
                                                                 
Total params: 127,703
Trainable params: 127,703
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
100/100 [=============================

In [13]:
loss, accuracy = model_dan.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

86/86 [==============================] - 0s 1ms/step - loss: 2.2521 - accuracy: 0.6784
Test loss: 2.2521, Test accuracy: 0.6784


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Assuming you have a list of sentences in 'sentences' and corresponding labels in 'labels'
# Make sure 'sentences' is a list of strings

# Tokenize and encode the sentences



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
encoded_sentences = tokenizer(input.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)


In [ ]:
label

0       2
1       1
2       1
3       2
4       2
       ..
2474    2
2475    2
2476    1
2477    2
2478    0
Name: experience, Length: 2479, dtype: int64

In [ ]:
print(encoded_sentences['input_ids'])

tensor([[  101,  1521,  1313,  ..., 29862, 29876,   102],
        [  101,   100,  1331,  ...,     0,     0,     0],
        [  101,   100,   100,  ...,     0,     0,     0],
        ...,
        [  101,  1328, 29859,  ...,     0,     0,     0],
        [  101,   100,  1329,  ...,     0,     0,     0],
        [  101,   100,  1328,  ...,     0,     0,     0]])


In [ ]:
labels_tensor = torch.tensor(label)

# Split data into train and test sets
encoded_train, encoded_test, y_train, y_test = train_test_split(encoded_sentences['input_ids'], labels_tensor, test_size=0.2, random_state=42)

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 5

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(**encoded_train, labels=y_train)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluate the model
model.eval()

with torch.no_grad():
    outputs = model(**encoded_test)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

# Calculate accuracy
accuracy = accuracy_score(y_test.numpy(), predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

TypeError: ignored

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have sentence embeddings in 'embedding_matrix' and labels in 'labels'
# Make sure 'embedding_matrix' is a 2D numpy array where each row corresponds to a sentence embedding

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embedding_matrix, labels, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

# Train the model
model.train()

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 5

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(input_ids=X_train_tensor, labels=y_train_tensor)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Evaluate the model
model.eval()

with torch.no_grad():
    outputs = model(input_ids=X_test_tensor)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")
